In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import math


test=pd.read_csv("C:/Users/HOME/Desktop/감귤착과량/test.csv")
train=pd.read_csv("C:/Users/HOME/Desktop/감귤착과량/train.csv")
submission = pd.read_csv("C:/Users/HOME/Desktop/감귤착과량/sample_submission.csv")

In [3]:
train['수관폭차이']=abs(train['수관폭2(max)']-train['수관폭1(min)'])
test['수관폭차이']=abs(test['수관폭2(max)']-test['수관폭1(min)'])

In [4]:
#새순 데이터만 걸러내기 위해 전처리
train1 = train[train.columns.drop(list(train.filter(regex='엽록소')))]
train1 = train1.drop(['ID','착과량(int)','수고(m)','수관폭1(min)','수관폭2(max)','수관폭평균','수관폭차이'], axis = 1)

In [13]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
x = train1.values
x = StandardScaler().fit_transform(x)
features = train.columns
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca_array = pca.fit_transform(x)
pca_df = pd.DataFrame(pca_array,columns = ["PCA"+str(i+1) for i in list(range(0,1))])
pca_df

,PCA1
0,11.958017
1,6.670798
2,10.042124
3,7.230745
4,0.622205
...,...
2202,-9.024632
2203,7.967929
2204,10.017448
2205,-0.360688


In [7]:
df_s=pd.concat([train['ID'],train.iloc[:,6:95]],axis=1)
new_cols = ['ID'] + [i for i in range(1, 90)]
df_s.columns = new_cols

In [8]:
from sklearn.linear_model import LinearRegression

coefs = []
for i in range(df_s.shape[0]):
    X = df_s.columns.values[1:].reshape(-1, 1)
    y = df_s.iloc[i, 1:].values
    reg_model = LinearRegression()
    reg_model.fit(X, y)
    coefs.append(reg_model.coef_[0])
print(coefs)

[-0.024669731018045617, -0.034405856315968666, -0.027557030983997272, -0.026785835886959476, -0.03474463738508682, -0.04277323799795708, -0.02606060606060605, -0.032836227442969004, -0.02493530813755532, -0.050585631596867534, -0.04117977528089886, -0.055917602996254666, -0.05359039836567925, -0.027781750085120863, -0.04578140960163431, -0.02931733060946544, -0.02548518896833503, -0.024572693224378616, -0.024005788219271358, -0.031181477698331628, -0.04765917602996254, -0.041511746680285996, -0.034514811031664955, -0.024051753489955727, -0.021736465781409593, -0.046731358529111325, -0.03795880149812733, -0.038946203609124944, -0.021727953694245825, -0.03195778004766768, -0.04914368403132447, -0.04874531835205991, -0.033743615934627165, -0.047298263534218586, -0.051411304051753474, -0.04369765066394279, -0.04240211099761661, -0.04251957780047667, -0.03359210078311201, -0.036624106230847796, -0.04054136874361592, -0.04386618998978548, -0.0476830098740211, -0.05269492679605037, -0.0437282

In [9]:
df_coefs = pd.DataFrame(coefs, columns=['기울기'])

# df_s와 df_coefs를 열 기준으로 합치기
df_s = pd.concat([df_s, df_coefs], axis=1)

In [10]:
df_s

,ID,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,기울기
0,TRAIN_0000,2.8,2.8,2.7,2.7,2.7,2.7,2.6,2.6,2.6,...,0.8,0.8,0.8,0.7,0.7,0.7,0.7,0.7,0.6,-0.024670
1,TRAIN_0001,3.3,3.3,3.3,3.2,3.2,3.1,3.1,3.1,3.0,...,0.7,0.6,0.6,0.6,0.5,0.0,0.0,0.0,0.0,-0.034406
2,TRAIN_0002,3.0,2.9,2.9,2.9,2.9,2.8,2.8,2.8,2.8,...,0.8,0.7,0.7,0.7,0.7,0.6,0.6,0.6,0.5,-0.027557
3,TRAIN_0003,3.1,3.0,3.0,3.0,3.0,2.9,2.9,2.9,2.8,...,0.9,0.9,0.8,0.8,0.8,0.8,0.7,0.7,0.7,-0.026786
4,TRAIN_0004,3.7,3.6,3.6,3.6,3.5,3.5,3.5,3.4,3.4,...,0.9,0.8,0.8,0.8,0.7,0.7,0.7,0.6,0.6,-0.034745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,TRAIN_2202,4.7,4.6,4.5,4.5,4.4,4.4,4.3,4.3,4.2,...,0.8,0.8,0.7,0.7,0.6,0.6,0.5,0.0,0.0,-0.048247
2203,TRAIN_2203,3.1,3.0,3.0,3.0,2.9,2.9,2.9,2.9,2.8,...,0.9,0.8,0.8,0.8,0.7,0.7,0.7,0.7,0.6,-0.027554
2204,TRAIN_2204,3.0,3.0,2.9,2.9,2.9,2.9,2.8,2.8,2.8,...,0.8,0.7,0.7,0.7,0.7,0.6,0.6,0.6,0.5,-0.027974
2205,TRAIN_2205,3.9,3.9,3.8,3.8,3.8,3.7,3.7,3.6,3.6,...,0.8,0.8,0.7,0.7,0.7,0.6,0.6,0.5,0.0,-0.039459


In [11]:
train['새순 기울기']=df_s['기울기']

In [14]:
train['새순 PCA']=pca_df['PCA1']

In [20]:
o=idx=2
train['새순mean'] = train.iloc[:,6:95].mean(axis=1)
train['새순std'] = train.iloc[:,6:95].std(axis=1)
train['새순min'] = train.iloc[:,6:95].min(axis=1)
train['새순max'] = train.iloc[:,6:95].max(axis=1)
train['새순gap'] = train.iloc[:,94] - train.iloc[:,6]
    
train['엽록소mean'] = train.iloc[:,95:184].mean(axis=1)
train['엽록소std'] = train.iloc[:,95:184].std(axis=1)
train['엽록소min'] = train.iloc[:,95:184].min(axis=1)
train['엽록소max'] = train.iloc[:,95:184].max(axis=1)
train['엽록소gap'] = train.iloc[:,183] - train.iloc[:,95] 

In [21]:
train

,ID,착과량(int),수고(m),수관폭1(min),수관폭2(max),수관폭평균,2022-09-01 새순,2022-09-02 새순,2022-09-03 새순,2022-09-04 새순,...,새순mean,새순std,새순min,새순max,새순gap,엽록소mean,엽록소std,엽록소min,엽록소max,엽록소gap
0,TRAIN_0000,692,275.0,287.0,292.0,289.5,2.8,2.8,2.7,2.7,...,1.693258,0.638143,0.6,2.8,-2.2,74.155836,2.427798,70.043251,78.336504,-8.293252
1,TRAIN_0001,534,293.0,284.0,336.0,310.0,3.3,3.3,3.3,3.2,...,1.861798,0.894491,0.0,3.3,-3.3,74.962123,2.813831,70.427600,80.116691,-9.689090
2,TRAIN_0002,634,300.0,392.0,450.0,421.0,3.0,2.9,2.9,2.9,...,1.762921,0.712611,0.5,3.0,-2.5,74.727999,2.476758,70.399578,79.118529,-8.718951
3,TRAIN_0003,639,289.0,368.0,379.0,373.5,3.1,3.0,3.0,3.0,...,1.857303,0.692802,0.7,3.1,-2.4,73.546210,2.641605,69.133970,77.936262,-8.802293
4,TRAIN_0004,496,306.0,353.0,358.0,355.5,3.7,3.6,3.6,3.6,...,2.125843,0.898171,0.6,3.7,-3.1,71.841067,2.642378,67.410093,76.233231,-8.823138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,TRAIN_2202,119,230.0,353.0,367.0,360.0,4.7,4.6,4.5,4.5,...,2.507865,1.248565,0.0,4.7,-4.7,69.822742,2.481985,65.560547,74.024614,-8.464066
2203,TRAIN_2203,611,266.0,337.0,400.0,368.5,3.1,3.0,3.0,3.0,...,1.841573,0.712519,0.6,3.1,-2.5,77.262155,2.681104,72.944396,81.920876,-8.976479
2204,TRAIN_2204,650,237.0,353.0,362.0,357.5,3.0,3.0,2.9,2.9,...,1.770787,0.723345,0.5,3.0,-2.5,71.165328,2.613637,66.479459,75.486518,-9.007058
2205,TRAIN_2205,363,281.0,337.0,403.0,370.0,3.9,3.9,3.8,3.8,...,2.184270,1.021129,0.0,3.9,-3.9,70.019605,2.376784,65.812084,74.110017,-8.297933


In [22]:
# 90개의 엽록소 칼럼을 PCA를 진행해 하나의 칼럼으로 요약

## 엽록소 칼럼 표준화
from sklearn.preprocessing import StandardScaler
train_scaled = StandardScaler().fit_transform(train.iloc[:,95:184])

In [23]:
## PCA 진행
from sklearn.decomposition import PCA
pca = PCA(n_components=1)  ### 하나로 축소
pca.fit(train_scaled)
train_pca = pca.transform(train_scaled)

In [24]:
## 엽록소(PCA) 데이터프레임 생성
pca_columns = ['pca_yup']
train_pca_yup = pd.DataFrame(train_pca, columns=pca_columns)
train_pca_yup.head()

,pca_yup
0,-3.814941
1,-5.671353
2,-5.130345
3,-2.415887
4,1.502893


In [25]:
train = train.drop(train.iloc[:,6:184],axis = 1)

In [28]:
train['엽록소 PCA']=train_pca_yup['pca_yup']

In [29]:
train

,ID,착과량(int),수고(m),수관폭1(min),수관폭2(max),수관폭평균,수관폭차이,새순 기울기,새순 PCA,새순mean,새순std,새순min,새순max,새순gap,엽록소mean,엽록소std,엽록소min,엽록소max,엽록소gap,엽록소 PCA
0,TRAIN_0000,692,275.0,287.0,292.0,289.5,5.0,-0.024670,11.958017,1.693258,0.638143,0.6,2.8,-2.2,74.155836,2.427798,70.043251,78.336504,-8.293252,-3.814941
1,TRAIN_0001,534,293.0,284.0,336.0,310.0,52.0,-0.034406,6.670798,1.861798,0.894491,0.0,3.3,-3.3,74.962123,2.813831,70.427600,80.116691,-9.689090,-5.671353
2,TRAIN_0002,634,300.0,392.0,450.0,421.0,58.0,-0.027557,10.042124,1.762921,0.712611,0.5,3.0,-2.5,74.727999,2.476758,70.399578,79.118529,-8.718951,-5.130345
3,TRAIN_0003,639,289.0,368.0,379.0,373.5,11.0,-0.026786,7.230745,1.857303,0.692802,0.7,3.1,-2.4,73.546210,2.641605,69.133970,77.936262,-8.802293,-2.415887
4,TRAIN_0004,496,306.0,353.0,358.0,355.5,5.0,-0.034745,0.622205,2.125843,0.898171,0.6,3.7,-3.1,71.841067,2.642378,67.410093,76.233231,-8.823138,1.502893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,TRAIN_2202,119,230.0,353.0,367.0,360.0,14.0,-0.048247,-9.024632,2.507865,1.248565,0.0,4.7,-4.7,69.822742,2.481985,65.560547,74.024614,-8.464066,6.142989
2203,TRAIN_2203,611,266.0,337.0,400.0,368.5,63.0,-0.027554,7.967929,1.841573,0.712519,0.6,3.1,-2.5,77.262155,2.681104,72.944396,81.920876,-8.976479,-10.956330
2204,TRAIN_2204,650,237.0,353.0,362.0,357.5,9.0,-0.027974,10.017448,1.770787,0.723345,0.5,3.0,-2.5,71.165328,2.613637,66.479459,75.486518,-9.007058,3.056180
2205,TRAIN_2205,363,281.0,337.0,403.0,370.0,66.0,-0.039459,-0.360688,2.184270,1.021129,0.0,3.9,-3.9,70.019605,2.376784,65.812084,74.110017,-8.297933,5.691486


In [30]:
train = train.drop(train.iloc[:,3:4],axis = 1)

In [31]:
train

,ID,착과량(int),수고(m),수관폭2(max),수관폭평균,수관폭차이,새순 기울기,새순 PCA,새순mean,새순std,새순min,새순max,새순gap,엽록소mean,엽록소std,엽록소min,엽록소max,엽록소gap,엽록소 PCA
0,TRAIN_0000,692,275.0,292.0,289.5,5.0,-0.024670,11.958017,1.693258,0.638143,0.6,2.8,-2.2,74.155836,2.427798,70.043251,78.336504,-8.293252,-3.814941
1,TRAIN_0001,534,293.0,336.0,310.0,52.0,-0.034406,6.670798,1.861798,0.894491,0.0,3.3,-3.3,74.962123,2.813831,70.427600,80.116691,-9.689090,-5.671353
2,TRAIN_0002,634,300.0,450.0,421.0,58.0,-0.027557,10.042124,1.762921,0.712611,0.5,3.0,-2.5,74.727999,2.476758,70.399578,79.118529,-8.718951,-5.130345
3,TRAIN_0003,639,289.0,379.0,373.5,11.0,-0.026786,7.230745,1.857303,0.692802,0.7,3.1,-2.4,73.546210,2.641605,69.133970,77.936262,-8.802293,-2.415887
4,TRAIN_0004,496,306.0,358.0,355.5,5.0,-0.034745,0.622205,2.125843,0.898171,0.6,3.7,-3.1,71.841067,2.642378,67.410093,76.233231,-8.823138,1.502893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,TRAIN_2202,119,230.0,367.0,360.0,14.0,-0.048247,-9.024632,2.507865,1.248565,0.0,4.7,-4.7,69.822742,2.481985,65.560547,74.024614,-8.464066,6.142989
2203,TRAIN_2203,611,266.0,400.0,368.5,63.0,-0.027554,7.967929,1.841573,0.712519,0.6,3.1,-2.5,77.262155,2.681104,72.944396,81.920876,-8.976479,-10.956330
2204,TRAIN_2204,650,237.0,362.0,357.5,9.0,-0.027974,10.017448,1.770787,0.723345,0.5,3.0,-2.5,71.165328,2.613637,66.479459,75.486518,-9.007058,3.056180
2205,TRAIN_2205,363,281.0,403.0,370.0,66.0,-0.039459,-0.360688,2.184270,1.021129,0.0,3.9,-3.9,70.019605,2.376784,65.812084,74.110017,-8.297933,5.691486
